In [21]:
# !pip install transforms
# !pip install sgmllib3k

In [22]:
import os
import numpy as np
import torch
from PIL import Image

`Faster R-CNN` is a model that predicts both bounding boxes and class scores for potential objects in the image.

In [4]:
class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms):
        self.root=root
        self.transforms = transforms
         #load all image file and sorting them and ensure the are align
        self.imgs = list(sorted(os.listdir(os.path(root,"PNGImage"))))
        self.masks = list(sorted(os.listdir(os.path(root,"PedMasks"))))
        
    def __getitem__(self,idx):
#         load image and mask
        img_path = os.path.join(self.root, "PNGImage", self.imgs[idx])
        mask_path = os.path.koin(self.root, "PedMask", self.masks[idx])
        img= Image.open(img_path).convert("RGB")
        
        """ note that we haven't converted the mask to RGB,
         because each color corresponds to a different instance
         with 0 being background"""
        mask =Image.open(mask_path)
        mask = np.array(mask)  #convert PIL image into numpy array
        obj_ids = np.unique(mask)  #first id is background so remove that
        obj_ids =obj_ids[1:]
        # split the color-encoded mask into a set
        # of binary masks
        masks=mask=obj_ids[:, None,None]
        
        #get boundaries and co-ordinate for each masks
        nums_obj =len(obj_ids)
        boxes =[]
        
        for i in range(nums_obj):
            pos= np.where(mask[i])
            xmin = np.min(pos[1])
            xmax=np.max(pos[1])
            ymin=np.min(pos[0])
            ymax= np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])
            
        #convert everything into torch tensors
        boxes=torch.as_tensor(boxes, dtype=torch.float32)
        labels= torch.ones((nums_obj), dtype=torch.int64)  #only have one classs
        masks=torch.as_tensor(masks, dtype=torch.uint8)
        
        image_id = torch.tensor([idx])
        area=(bbox[:, 3]- boxes[:, 1]) * (boxes[:, 2]- boxes[:,0])
        #suppose all instances are not croud
        iscrowd =torch.zeros((nums_obj),dtype=torch.int64)
        
        target= {}
        target["boxes"]=boxes
        target["labels"]=labels
        target["masks"] =masks
        target["image_id"] =image_id
        target["area"] =area
        target["iscrowd"] = iscrowd
        
        if self.transforms is not None:
            img, target = self.transforms(img,target)
            
        return img, target
    
    def __len__(self):
        return len(self.imgs)
    
#thats all for this dataset now make a model which can work on this datasets

####  1 - Finetuning from a pretrained model

In [7]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [8]:
#load a model pre_train_coco
model =torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to C:\Users\gazur/.cache\torch\hub\checkpoints\fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100.0%


In [12]:
# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes=2  # 1 class (person) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor=FastRCNNPredictor(in_features,num_classes)

##### 2 - Modifying the model to add a different backbone

In [13]:
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

In [14]:
"""understand and written"""

# load a pre-trained model for classification and return
# only the features
backbone = torchvision.models.mobilenet_v2(pretrained=True).features
# FasterRCNN needs to know the number of
# output channels in a backbone. For mobilenet_v2, it's 1280
# so we need to add it here
backbone.out_channels = 1280

# let's make the RPN generate 5 x 3 anchors per spatial
# location, with 5 different sizes and 3 different aspect
# ratios. We have a Tuple[Tuple[int]] because each feature
# map could potentially have different sizes and
# aspect ratios
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))

# let's define what are the feature maps that we will
# use to perform the region of interest cropping, as well as
# the size of the crop after rescaling.
# if your backbone returns a Tensor, featmap_names is expected to
# be [0]. More generally, the backbone should return an
# OrderedDict[Tensor], and in featmap_names you can choose which
# feature maps to use.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                output_size=7,
                                                sampling_ratio=2)

# put the pieces together inside a FasterRCNN model
model = FasterRCNN(backbone,
                   num_classes=2,
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\gazur/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth
100.0%


In [15]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [20]:
import transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [30]:
!pip install pyttsx3

     -------------------------------------- 167.4/167.4 kB 1.7 MB/s eta 0:00:00


In [31]:
from engine import train_one_epoch, evaluate
import utils


def main():
    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # our dataset has two classes only - background and person
    num_classes = 2
    # use our dataset and defined transformations
    dataset = PennFudanDataset('PennFudanPed', get_transform(train=True))
    dataset_test = PennFudanDataset('PennFudanPed', get_transform(train=False))

    # split the dataset in train and test set
    indices = torch.randperm(len(dataset)).tolist()
    dataset = torch.utils.data.Subset(dataset, indices[:-50])
    dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

    # define training and validation data loaders
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2, shuffle=True, num_workers=4,
        collate_fn=utils.collate_fn)

    data_loader_test = torch.utils.data.DataLoader(
        dataset_test, batch_size=1, shuffle=False, num_workers=4,
        collate_fn=utils.collate_fn)

    # get the model using our helper function
    model = get_model_instance_segmentation(num_classes)

    # move model to the right device
    model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)

    # let's train it for 10 epochs
    num_epochs = 10

    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
        evaluate(model, data_loader_test, device=device)

    print("That's it!")

ImportError: cannot import name 'train_one_epoch' from 'pyttsx3' (C:\Users\gazur\Anaconda3\envs\Pytorch\lib\site-packages\pyttsx3\__init__.py)